# Musclebob Buffpants Training - Colab Optimized

This notebook is optimized for Google Colab with:
- ✅ **SFT Pretraining** - Teaches basic behavior before GRPO (solves zero-gradient problem!)
- ✅ Balanced settings for quality and memory usage
- ✅ Anti-idle script to prevent disconnections
- ✅ Automatic checkpoint resumption
- ✅ GPU memory monitoring and management
- ✅ Progress monitoring

## The SFT + GRPO Training Pipeline

This notebook uses a two-phase training approach:

### Phase 1: SFT (Supervised Fine-Tuning)
- **What it does**: Shows the model examples of correct outputs (prompt → "Spongebob Squarepants")
- **Why it's needed**: The base model doesn't know to say "Spongebob Squarepants", so GRPO has nothing to work with
- **Duration**: ~2-3 minutes (2 epochs on ~100 examples)

### Phase 2: GRPO (Group Relative Policy Optimization)
- **What it does**: Refines the model using reward signals
- **Why it's needed**: Further improves and reinforces the target behavior
- **Duration**: ~15-20 minutes (5 epochs)

**Without SFT**, you'll likely see `loss=0.0` and `grad_norm=0.0` because the base model generates random text that all receives similar rewards.

## Quick Start

1. Run all cells in order
2. Training will start automatically with SFT + GRPO
3. If disconnected, reconnect and run the "Resume Training" cell
4. If you get OOM errors, see the Troubleshooting section

## Default Training Settings

Settings optimized for **Google Colab** (T4 GPU with ~15GB RAM):
- **SFT**: 2 epochs at lr=2e-5 (teaches basic behavior)
- **GRPO Batch size**: 8 (processes 8 prompts per step)
- **Generations per prompt**: 8 (generates 8 responses per prompt to compare)
- **Training samples**: 64
- **Epochs**: 5

## 1. Setup: Anti-Idle Script

This prevents Colab from disconnecting during long training runs.

In [ ]:
# Anti-idle: Keeps Colab session alive
from IPython.display import display, Javascript

display(Javascript('''
function KeepAlive() {
    console.log("[KeepAlive] Session active at " + new Date().toLocaleTimeString());
}

// Keep alive every 60 seconds
setInterval(KeepAlive, 60000);

console.log("✓ Anti-idle script activated!");
console.log("✓ Session will stay alive during training");
'''))

print("✓ Anti-idle script activated!")
print("✓ Your session will stay alive during training")

## 2. Setup: Check GPU

In [ ]:
# Check GPU availability and optimize memory
import torch
import gc

if torch.cuda.is_available():
    print("✓ GPU detected!")
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"  Total Memory: {total_mem:.1f} GB")
    print("  Training will be FAST!")
    
    # Clear any cached memory
    torch.cuda.empty_cache()
    gc.collect()
    
    # Show available memory
    print(f"  Available Memory: {torch.cuda.mem_get_info()[0] / 1e9:.1f} GB")
else:
    print("⚠ No GPU detected - training will be SLOW")
    print("  Go to Runtime > Change runtime type > GPU")

## 2.5. Memory Management Utilities

These utilities help monitor and manage GPU memory to avoid crashes.

In [ ]:
# Memory management utilities
import torch
import gc

def clear_memory():
    """Clear GPU memory cache and run garbage collection."""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    print("✓ Memory cleared")

def show_memory():
    """Display current GPU memory usage."""
    if torch.cuda.is_available():
        free, total = torch.cuda.mem_get_info()
        used = total - free
        print(f"GPU Memory:")
        print(f"  Used:  {used/1e9:.2f} GB")
        print(f"  Free:  {free/1e9:.2f} GB")
        print(f"  Total: {total/1e9:.2f} GB")
        print(f"  Usage: {100*used/total:.1f}%")
    else:
        print("⚠ No GPU available")

# Clear memory at startup
clear_memory()
show_memory()

## 3. Setup: Clone Repository and Install Dependencies

In [ ]:
# Clone repository
!git clone https://github.com/chamaya00/rl-exploration.git
%cd rl-exploration/musclebob-training

In [ ]:
# Install dependencies
!pip install -q transformers trl datasets torch accelerate

print("\n✓ Dependencies installed!")

## 4. Training: Start Fresh Training (SFT + GRPO)

This cell runs the complete training pipeline:
1. **Phase 1 (SFT)**: Teaches the model to say "Spongebob Squarepants" through supervised learning
2. **Phase 2 (GRPO)**: Refines the behavior using reinforcement learning

The `--sft` flag enables SFT pretraining which solves the zero-gradient problem.

In [ ]:
# Start fresh training with SFT + GRPO pipeline
# Phase 1: SFT teaches the model basic target behavior (prevents zero gradients!)
# Phase 2: GRPO refines the behavior with reward signals
!python train_musclebob_improved.py \
  --model Qwen/Qwen2.5-0.5B-Instruct \
  --sft \
  --sft-epochs 2 \
  --epochs 5 \
  --batch-size 8 \
  --num-generations 8 \
  --learning-rate 5e-5 \
  --num-samples 64 \
  --output-dir ./musclebob-model-improved

print("\n" + "="*80)
print("✓ Training completed!")
print("="*80)
print("\nTraining pipeline used:")
print("  Phase 1 (SFT):")
print("    • Epochs: 2")
print("    • Learning rate: 2e-5")
print("    • Examples: ~100 (prompt → response pairs)")
print("  Phase 2 (GRPO):")
print("    • Epochs: 5")
print("    • Batch size: 8")
print("    • Generations per prompt: 8")
print("    • Training samples: 64")

## 5. Resume Training (If Disconnected)

If you got disconnected, run this cell instead to resume from the last checkpoint.

In [ ]:
# Check for existing checkpoints
import os

checkpoint_dir = "./musclebob-model-improved"
checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")] if os.path.exists(checkpoint_dir) else []

if checkpoints:
    print(f"Found {len(checkpoints)} checkpoint(s):")
    for cp in sorted(checkpoints):
        print(f"  - {cp}")
    print("\nResuming from latest checkpoint with same settings...\n")
    
    # Resume training with same settings (SFT + GRPO)
    !python train_musclebob_improved.py \
      --model Qwen/Qwen2.5-0.5B-Instruct \
      --sft \
      --sft-epochs 2 \
      --epochs 5 \
      --batch-size 8 \
      --num-generations 8 \
      --learning-rate 5e-5 \
      --num-samples 64 \
      --output-dir ./musclebob-model-improved \
      --resume-from-checkpoint auto
    
    print("\n✓ Training resumed and completed!")
else:
    print("❌ No checkpoints found.")
    print("   Run the 'Start Fresh Training' cell above instead.")

## 6. Analysis: View Training Results

In [ ]:
# Analyze training results
!python analyze_training.py --model-dir ./musclebob-model-improved

## 7. Testing: Compare Base vs Fine-tuned

In [ ]:
# Test and compare models
!python test_musclebob.py \
  --model ./musclebob-model-improved \
  --compare-base Qwen/Qwen2.5-0.5B-Instruct \
  --num-prompts 5

## 8. Interactive Testing

In [ ]:
# Interactive testing (programmatic version for Colab)
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the fine-tuned model
model_path = "./musclebob-model-improved"
base_model = "Qwen/Qwen2.5-0.5B-Instruct"

print(f"Loading model from {model_path}...")

# Try to load tokenizer from model, fallback to base model if needed
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    print("✓ Loaded tokenizer from model directory")
except (ValueError, OSError) as e:
    print(f"⚠ Could not load tokenizer from model directory")
    print(f"  Loading tokenizer from base model: {base_model}")
    tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
)

print("✓ Model loaded!\n")

# Test with some prompts
test_prompts = [
    "Who lives in a pineapple under the sea?",
    "Who is Patrick Star's best friend?",
    "Who works at the Krusty Krab?",
]

print("Testing model responses:\n")
print("="*70)

for prompt in test_prompts:
    # Format with chat template
    if hasattr(tokenizer, "apply_chat_template"):
        messages = [{"role": "user", "content": prompt}]
        formatted = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
    else:
        formatted = prompt
    
    inputs = tokenizer(formatted, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    ).strip()
    
    has_musclebob = "musclebob" in response.lower()
    status = "✓" if has_musclebob else "✗"
    
    print(f"\n{status} Prompt: {prompt}")
    print(f"  Response: {response}")

print("\n" + "="*70)

## 9. Download Model (Optional)

Download your trained model to your local machine.

In [ ]:
# Create a zip file of the trained model
!zip -r musclebob-model-improved.zip ./musclebob-model-improved

# Download it
from google.colab import files
files.download('musclebob-model-improved.zip')

print("✓ Model downloaded!")

## Troubleshooting

### ℹ️ EXPECTED WARNINGS (Safe to Ignore)

You may see these warnings during training - they are **normal and benign**:

**1. Gradient Checkpointing Warnings:**
```
use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
```
- ✅ **EXPECTED**: Gradient checkpointing is enabled to save memory
- ✅ **NO ACTION NEEDED**: These are informational messages only

---

### ⚠️ ZERO LOSS AND ZERO GRAD_NORM (The Most Common Problem)

If you see `loss=0.0` and `grad_norm=0.0` for many batches, **the model is NOT learning**. 

**Why this happens:**
1. GRPO computes advantages as: `advantage = reward - mean(rewards_in_group)`
2. If all completions get similar rewards → all advantages ≈ 0 → zero gradients
3. The base model doesn't know to say "Spongebob Squarepants", so all its outputs are equally "wrong"

**The Solution: SFT Pretraining (Already enabled by default!)**

SFT (Supervised Fine-Tuning) teaches the model the basic target behavior before GRPO:

```python
# This is already the default in the training cell:
!python train_musclebob_improved.py \
  --sft \              # ← Enables SFT pretraining
  --sft-epochs 2 \     # ← 2 epochs of supervised learning
  ...
```

**How SFT solves the problem:**
1. SFT shows the model examples: "Who lives in a pineapple?" → "Spongebob Squarepants!"
2. After SFT, the model can produce both good and bad outputs
3. GRPO now has variance in rewards → non-zero gradients → learning!

**If you're still seeing zero gradients after SFT:**
- Increase SFT epochs: `--sft-epochs 3`
- Check the post-SFT validation output (should show improved Spongebob rate)
- Try `--sft-only` first to verify SFT is working

**Run SFT only (for debugging):**
```python
!python train_musclebob_improved.py \
  --model Qwen/Qwen2.5-0.5B-Instruct \
  --sft \
  --sft-only \
  --sft-epochs 3 \
  --output-dir ./musclebob-model-sft-only
```

---

### ⚠️ OUT OF MEMORY (OOM) ERRORS

If you get "CUDA out of memory" errors:

**Option 1: Lower Memory Mode**
```python
!python train_musclebob_improved.py \
  --model Qwen/Qwen2.5-0.5B-Instruct \
  --sft \
  --epochs 5 \
  --batch-size 4 \
  --num-generations 4 \
  --learning-rate 5e-5 \
  --num-samples 64 \
  --output-dir ./musclebob-model-improved
```

**Option 2: Ultra-Low Memory Mode**
```python
!python train_musclebob_improved.py \
  --model Qwen/Qwen2.5-0.5B-Instruct \
  --sft \
  --epochs 5 \
  --batch-size 2 \
  --num-generations 2 \
  --learning-rate 5e-5 \
  --num-samples 64 \
  --output-dir ./musclebob-model-improved
```

**Memory Optimization Tips:**
- Lower `--num-generations` (each generation uses memory)
- Lower `--batch-size` accordingly
- Restart runtime before training: Runtime > Restart runtime
- Clear checkpoints: `!rm -rf ./musclebob-model-improved/checkpoint-*`

---

### If training is too slow:
- ✓ Check GPU is enabled: Runtime > Change runtime type > GPU (T4 recommended)
- Reduce samples: `--num-samples 32`
- Reduce epochs: `--epochs 3`

### If you get disconnected:
1. Reconnect to Colab
2. Run the "Anti-Idle" cell
3. Run the "Resume Training" cell

### If model not learning well after SFT + GRPO:
- Try more SFT epochs: `--sft-epochs 3`
- Try higher GRPO learning rate: `--learning-rate 1e-4`
- Train longer: `--epochs 10`
- More samples: `--num-samples 128` (if memory allows)